The table given by the driver has only 7 attributes. But we may also mining some interesting information about the driver. Let's try it step by step...

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pylab import *
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

##1. Overal Looking...

Let's load and look at the data.

In [ ]:
df=pd.read_csv("../input/My Uber Drives - 2016.csv")
print("Head:\n",df.head(10))
print("Tail:\n",df.tail(10))

Pay attention to the last line of the table. It is the "Totals", not a specific driving content. So, we could ignore the last line at first.

In [ ]:
df=df[:-1]

##2. Single Attribute Analysis...

###A. START_DATE

Let's analyze the START_DATE first.
We will see the driving distribute by month and by hour.

**By Month**

In [ ]:
df["START_DATE*"]=pd.to_datetime(df["START_DATE*"],format="%m/%d/%Y %H:%M")
sd_m_dis=df["START_DATE*"].dt.month.value_counts()
sd_m_dis=sd_m_dis.sort_index()
sd_m_mean=sd_m_dis.mean()
print("Month Distribute:",sd_m_dis)
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x()+rect.get_width()/2., 1.03*height, '%s' % int(height))
figure(0)
rects=plt.bar(sd_m_dis.index,sd_m_dis.values)
plt.plot([0,len(sd_m_dis.index)+1],[sd_m_mean,sd_m_mean],"r--")
plt.title("START_DATE DISTRIBUTE")
plt.xlabel("Month")
plt.ylabel("Driving Times")
plt.grid()
autolabel(rects)
plt.savefig("./sd_m_dis_fig")

As we can see from the figure. The drive seemed drive more in December, August. On the contrary, he drive less in September, May, April and January. Did he have anything important during the months?

**By Hour**

In [ ]:
sd_h_dis=df["START_DATE*"].dt.hour.value_counts()
sd_h_dis=sd_h_dis.sort_index()
print("Hours Distribute:",sd_h_dis)
sd_h_mean=sd_h_dis.mean()
figure(1)
rects=plt.bar(sd_h_dis.index,sd_h_dis.values)
plt.plot([0,len(sd_h_dis.index)+1],[sd_h_mean,sd_h_mean],"r--")
plt.title("START_DATE(Hours) DISTRIBUTE")
plt.xlabel("Hours")
plt.ylabel("Driving Times")
plt.grid()
autolabel(rects)
plt.savefig("./sd_h_dis_fig")

Let's look at the distribute by hour. Most orders are started between 10:00 a.m. to 6:00 p.m.  The life is regular, isn't it?

###B. END_DATE

END_DATE is similar to START_DATE. So we pass it...

###C. CATEGORY

In [ ]:
ct_dis=df["CATEGORY*"].value_counts()
figure(2)
rects=plt.bar(range(1,len(ct_dis.index)+1),ct_dis.values)
plt.title("Category DISTRIBUTE")
plt.xlabel("Category")
plt.ylabel("Quantity")
plt.xticks(range(1,len(ct_dis.index)+1),ct_dis.index)
plt.grid()
autolabel(rects)
plt.savefig("./ct_dis_fig")

As we can see, most of the driving jobs are Business Type. So, the driver may be a "business-oriented" driver or the business events happened just as the driver began to work. We will analyze it later.

###D. START

In [ ]:
st_dis=df["START*"].value_counts()
st_dis.sort_values(inplace=True,ascending=False)
st_dis=st_dis.iloc[:10]
print("START_PLACE:\n",st_dis)

Is the driver's home near Cary? 

###E. STOP

In [ ]:
stp_dis=df["STOP*"].value_counts()
stp_dis.sort_values(ascending=False)
stp_dis=stp_dis.iloc[:10]
print("STOP_PLACE:\n",stp_dis)

We can confirm that the driver's "heritage" is in Cary(Is this a district?)

###F. MILES

In [ ]:
ml_dis=df["MILES*"]
ml_range_lst=["<=5","5-10","10-15","15-20",">20"]
ml_dic=dict()
for item in ml_range_lst:
    ml_dic[item]=0
for mile in ml_dis.values:
    if mile<=5:
        ml_dic["<=5"]+=1
    elif mile<=10:
        ml_dic["5-10"]+=1
    elif mile<=15:
        ml_dic["10-15"]+=1
    elif mile<=20:
        ml_dic["15-20"]+=1
    else:
        ml_dic[">20"]+=1
ml_dis=pd.Series(ml_dic)
ml_dis.sort_values(inplace=True,ascending=False)
print("Miles:\n",ml_dis)
figure(6)
rects=plt.bar(range(1,len(ml_dis.index)+1),ml_dis.values)
plt.title("Miles")
plt.xlabel("Miles")
plt.ylabel("Quantity")
plt.xticks(range(1,len(ml_dis.index)+1),ml_dis.index)
plt.grid()
autolabel(rects)
plt.savefig("./ml_dis_fig")

Maybe the driver likes short-distance driving OR the the short-distance driving tasks is relatively more than the long-distance tasks. We need more data to get the conclusion.

###G. PURPOSE

In [ ]:
pp_dis=df["PURPOSE*"].value_counts()
pp_dis.sort_values(ascending=False)
pp_dis=pp_dis.iloc[:10]
print("PURPOSE:\n",pp_dis)

The Meeting purpose and the Meal/Entertain task the lead.  We need to analyze the PURPOSE with the other attributes.

##2. Association Analysis

Next, let's do the association analysis.
We could regard every single attribute as a single element. The association analysis needs that the different attribute mustn't have the same value and that the values must be discrete. So, we have to do something first. 

In [ ]:
#Relational Analysis：SSupport & Confidence
#Rebuild the df.
new_df=df.copy()
new_df["START*"]=["START_"+str(s) for s in df["START*"].values]
new_df["STOP*"]=["STOP_"+str(s) for s in df["STOP*"].values]
new_df["CATEGORY*"]=["CAT_"+str(s) for s in df["CATEGORY*"].values]
new_df["START_MONTH"]=["M_"+str(s) for s in df["START_DATE*"].dt.month.values]
new_df["START_HOUR"]=["H_"+str(s) for s in df["START_DATE*"].dt.hour.values]
tsf_miles=[]
for mile in df["MILES*"].values:
    if mile < 5.0:
        tsf_miles.append("<5")
    elif mile < 10.0:
        tsf_miles.append("5-10")
    elif mile < 15.0:
        tsf_miles.append("10-15")
    elif mile < 20.0:
        tsf_miles.append("15-20")
    else:
        tsf_miles.append(">20")
new_df["MILES_RANGE"]=pd.Series(tsf_miles)
print(new_df["START_DATE*"])
new_df.drop("START_DATE*",axis=1,inplace=True)
new_df.drop("END_DATE*",axis=1,inplace=True)
new_df.drop("MILES*",axis=1,inplace=True)
print(new_df)
print(new_df.shape)

We don't need "Unknown" and "nan" attribute. So...

In [ ]:
apr_itms=[]
for item in new_df.values:
    item=list(item)
    if str(item[3])=="nan":
        item.remove(item[3])
    if "Unknown" in item[2]:
        item.remove(item[2])
    if "Unknown" in item[1]:
        item.remove(item[1])
    apr_itms.append(item)

Then, with the Apriori Algorithm. We could do the analysis.
First, let's construct the Objects.

In [ ]:
#Support&Confidence
from itertools import combinations
def comb(lst):
    ret=[]
    for i in range(1,len(lst)+1):
        ret+=list(combinations(lst,i))
    return ret
class AprLayer(object):
    d=dict()
    def __init__(self):
        self.d=dict()
class AprNode(object):
    s=set()
    size=0
    num=0
    lnk_nodes=dict()
    def __init__(self,node):
        self.s=set(node)
        self.size=len(self.s)
        self.lnk_nodes=dict()
        self.num=0
    def __hash__(self):
        return hash("__".join(sorted([str(itm) for itm in list(self.s)])))
    def __eq__(self, other):  
        if "__".join(sorted([str(itm) for itm in list(self.s)]))=="__".join(sorted([str(itm) for itm in list(other.s)])):  
            return True  
        return False
    def isSubnode(self,node):
        return self.s.issubset(node.s)
    def incNum(self,num=1):
        self.num+=num
    def addLnk(self,node):
        self.lnk_nodes[node]=node.s

class AprBlk():
    apr_layers=dict()
    data_num=0
    def __init__(self,data):
        cnt=0
        self.data_num=len(data)
        for datum in data:
            cnt+=1
            datum=comb(datum)
            nodes=[AprNode(da) for da in datum]
            for node in nodes:
                if not node.size in self.apr_layers:
                    self.apr_layers[node.size]=AprLayer()
                if not node in self.apr_layers[node.size].d:
                    self.apr_layers[node.size].d[node]=node
                self.apr_layers[node.size].d[node].incNum()
            for node in nodes:
                if node.size==1:
                    continue
                for sn in node.s:
                    sub_n=AprNode(node.s-set([sn]))
                    self.apr_layers[node.size-1].d[sub_n].addLnk(node)
            #        
            #max_size=max(list(self.apr_layers.keys()))
            #for i in range(1,max_size):
            #    for subnode in self.apr_layers[i].d:
            #        for bignode in self.apr_layers[i+1].d:
            #            if subnode.isSubnode(bignode):
            #                subnode.addLnk(bignode)
    def getFreqItems(self,thd=10,hd=1):
        freq_items=[]
        for layer in self.apr_layers:
            for node in self.apr_layers[layer].d:
                if self.apr_layers[layer].d[node].num<thd:
                    continue
                freq_items.append((self.apr_layers[layer].d[node].s,self.apr_layers[layer].d[node].num))
        freq_items.sort(key=lambda x:x[1],reverse = True)
        return freq_items[:hd]
    def getLowConf(self,h_thd=10,l_thd=1,hd=1):
        low_confidence=[]
        for layer in self.apr_layers:
            for node in self.apr_layers[layer].d:
                if self.apr_layers[layer].d[node].num<h_thd:
                    continue
                for lnk_node in node.lnk_nodes:
                    if lnk_node.num<l_thd:
                        continue
                    conf=float(lnk_node.num)/float(node.num)
                    low_confidence.append([node.s,node.num,lnk_node.s,lnk_node.num,conf])
        low_confidence.sort(key=lambda x:x[4])
        return low_confidence[:hd]

Then, we could find the frequent item-sets.

In [ ]:
apr_blk=AprBlk(apr_itms)
freq_items=apr_blk.getFreqItems(thd=10,hd=30)
print(freq_items)
figure(6)
freq_x=["_".join(sorted(list(s[0]))) for s in freq_items]
freq_y=[s[1] for s in freq_items]
plt.bar(range(1,len(freq_items)+1),freq_y)
plt.title("Freq Items")
plt.xlabel("Items")
plt.ylabel("Quantity")
#plt.xticks(range(1,len(freq_x)+1),freq_x)
plt.grid()
#autolabel(rects)
plt.savefig("./ass_ana_fig")

We could also get the low confident item-set.

In [ ]:
print(apr_blk.getLowConf(h_thd=10,l_thd=10,hd=30))

From the results, we could see which attributes have high level correlation and which attribute influence the driver's  driving behavior or some other information about the passengers.

For example,{ '<5', 'CAT_Business'} has the high support level. So we could conclude that the distance of most "Business Category" orders is less than 5 miles.
 
TO BE CONTINUED...

##3. Others

Let's concentrate on the speed.
First, let's get them(Miles/min).

In [ ]:
df["END_DATE*"]=pd.to_datetime(df["END_DATE*"],format="%m/%d/%Y %H:%M")
speed=df["MILES*"]/((df["END_DATE*"]-df["START_DATE*"]).dt.seconds/60)
print(speed)

Then let's group them by the start hour. Let's see the result.

In [ ]:
df["SPEED*"]=speed
df["START_HOUR*"]=df["START_DATE*"].dt.hour
print(df[df["SPEED*"]!=np.inf])
spd_df=df[df["SPEED*"]!=np.inf].groupby(["START_HOUR*"])["SPEED*"].mean()
print(spd_df)
figure(10)
#rects=plt.bar(range(0,len(spd_df.index)),spd_df.values)
rects=plt.bar(spd_df.index,spd_df.values)
plt.title("Speed")
plt.xlabel("Time(Hour)")
plt.ylabel("Speed[Mile(s)/min]")
plt.xticks(spd_df.index)
plt.grid()
plt.savefig("./speed_fig")

From the bar chart, we could see that the speed during 7:00-9:59 & 13:00-16:59 was relatively low. Maybe it was easy to meet a jam on the road. The driver certainly went home before 4:00. Why did the driver's car run so slowly during 0:00-1:00? Was the night life so abundant?